In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
import getpass
import os

os.environ["NVIDIA_API_KEY"] = getpass.getpass()

from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="meta/llama3-70b-instruct")

 ········


In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'role': 'assistant', 'content': "Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 15, 'total_tokens': 41, 'completion_tokens': 26}, 'finish_reason': 'stop', 'model_name': 'meta/llama3-70b-instruct'}, id='run-b5d9b4e2-ebfe-4c50-936e-7519771647c4-0', role='assistant')

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't know your name. I'm a large language model, I don't have access to personal information or memories, and I don't recall any previous conversations with you. Each time you interact with me, it's a new conversation. Would you like to share your name with me?", response_metadata={'role': 'assistant', 'content': "I'm sorry, but I don't know your name. I'm a large language model, I don't have access to personal information or memories, and I don't recall any previous conversations with you. Each time you interact with me, it's a new conversation. Would you like to share your name with me?", 'token_usage': {'prompt_tokens': 15, 'total_tokens': 80, 'completion_tokens': 65}, 'finish_reason': 'stop', 'model_name': 'meta/llama3-70b-instruct'}, id='run-864b623c-2e12-47db-bd8e-00aa7b9ea014-0', role='assistant')

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I remember! Your name is Bob!', response_metadata={'role': 'assistant', 'content': 'I remember! Your name is Bob!', 'token_usage': {'prompt_tokens': 40, 'total_tokens': 49, 'completion_tokens': 9}, 'finish_reason': 'stop', 'model_name': 'meta/llama3-70b-instruct'}, id='run-2342dee6-b644-4051-8442-4c98f279b343-0', role='assistant')

In [7]:
!pip install langchain_community

In [8]:
from langchain_core.chat_history import (BaseChatMessageHistory,InMemoryChatMessageHistory,)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {"configurable":{"session_id":"abc2"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)
response.content

"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I remember! Your name is Bob!'

In [12]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't know your name. I'm a large language model, I don't have access to personal information or memories, and I don't recall any previous conversations with you. Each time you interact with me, it's a new conversation. Would you like to share your name with me?"

In [13]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I know this one! Your name is Bob!'

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability",
        ),
        MessagesPlaceholder(variable_name="message"),
        
    ]
)
chain = prompt | model

In [15]:
response = chain.invoke({"message":[HumanMessage(content="hi!I am bob")]})
response.content

"Hi Bob! It's nice to meet you! Is there something I can help you with or would you like to chat about something in particular? I'm all ears!"

In [16]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [17]:
config = {"configurable":{"session_id":"abc5"}}

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I am jim")],
    config=config,
)
response.content

"Hi Jim! It's nice to meet you. I'm happy to assist you with any questions or topics you'd like to discuss. What's on your mind today? Do you have a specific question or topic you'd like to explore? I'm all ears!"

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I remember! Your name is Jim!'

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [33]:
response = chain.invoke(
    { "language": "Chinese","messages": [HumanMessage(content="hi! I'm bob")]}
)

response.content

'(nǐ hǎo, bó bó) - Hello, Bob!'

In [41]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_message_key="messages",
)

In [42]:
config = {"configurable": {"session_id": "abc11"}}

In [47]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 789.8 kB/s eta 0:00:00 0:00:01
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.7.24-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached safetensors-0.4.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 20.6 MB/s eta 0:00:0000:0100:01
Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
Using cached regex-2024.7.24-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (786 kB)
Using cached safetensors-0.4.3-cp311-cp311-manylinux_2_

In [52]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [53]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't know your name. You didn't tell me your name. I'm just a helpful assistant, and I don't have any information about you."

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is!'

In [55]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't know your name. We just started chatting, and you haven't told me your name yet! Would you like to share it with me?"

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem yet! This is the start of our conversation, and I'm ready to help you with any question you have. What would you like to ask or discuss?"

In [58]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


|Hi| Todd|!| Here|'s| a| joke| for| you|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| brought| a| smile| to| your| face|,| Todd|!||